In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

In [ ]:
import numpy as np
from loader.CSVLoader import csv_loader

gt,wave = csv_loader()

#Load data for every element
name="example"

all_data_k = np.load(f"./experiment/{name}_K.npy")
test_data_k = np.load(f'./experiment/{name}_K_test.npy')

all_data_p = np.load(f"./experiment/{name}_P.npy")
test_data_p = np.load(f'./experiment/{name}_P_test.npy')

all_data_Mg = np.load(f"./experiment/{name}_Mg.npy")
test_data_Mg = np.load(f'./experiment/{name}_Mg_test.npy')

all_data_pH = np.load(f"./experiment/{name}_pH.npy")
test_data_pH = np.load(f'./experiment/{name}_pH_test.npy')

In [ ]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.ModelParams import Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams
from creators.CNN2DCreator import  CNN2DNetworkParams
from prep.HelperBlocks import pipe

from prep.Normlize import normalize_layers

drop=0.5
epochs=50
optimizer = tf.keras.optimizers.Adam
folds = 4

#define a model using any Params from './creators'
params_cnn2d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn3d = CNN3DNetworkParams(
    conv=[Conv3DParams(kernel=(10,10,1),pool_size=(10,10,1),filters=32,normalization=True,activation="relu", dropout=drop),
          Conv3DParams(kernel=(1,1,15),pool_size=(1,1,15),filters=128, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: np.expand_dims(d, axis=-1),
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

In [ ]:
from ModelExperiment import ModelExperiment

#run the experiment using data+models
experiment = ModelExperiment()
experiment.run_experiment(
    params_vector=[params_cnn2d,params_cnn3d,params_cnn2d,params_cnn3d],
    data=[all_data_k,all_data_k,all_data_p,all_data_p],
    gt=[gt["K"].values,gt["K"].values,gt["P"].values,gt["P"].values],
    folds=folds,
    test_data=[test_data_k,test_data_k,test_data_p,test_data_p]
)

In [ ]:
#save history of a selected model
experiment.models[0].save_history('./history1.csv')

In [ ]:
#get errors report
experiment.get_errors_report('./errors1.csv')

In [ ]:
#read times of operations
print(experiment.models[0].last_crossval_predict_time)
print(experiment.models[0].last_crossval_fit_time)

In [ ]:
#read statistics
print(experiment.models[0].crossval_rs)